<a href="https://colab.research.google.com/github/Ksh1t1zsharma/Netflix-Movies-and-TV-Shows-Clustering/blob/main/Multiclass_Fish_Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Multiclass Fish Image Classification**

##### **Project Type**    - Classification
##### **Contribution**    - Individual

# **Project Summary**

- This project, titled Multiclass Fish Image Classification, focuses on developing a robust deep learning pipeline to accurately classify images of fish into multiple species. The goal is to compare different model architectures, identify the most effective one, and prepare the solution for deployment through a Streamlit-based web application.
- The dataset comprises images of various fish species organized into training, validation, and testing folders. Each subfolder within these sets corresponds to a distinct fish class. The data was preprocessed using TensorFlow’s ImageDataGenerator, applying rescaling to normalize pixel values to the range and employing augmentation techniques such as random rotations, shifts, shears, zooms, and horizontal flips. This augmentation step aimed to improve model generalization and handle potential overfitting.
- An extensive Exploratory Data Analysis (EDA) was conducted which included:
 - Count plots & pie charts to understand the distribution of classes.
 - Sample image display to visually validate labeling correctness.
 - Dimension and color channel checks to ensure data consistency.
 - Brightness and average color variation analysis to explore visual differences between species.
- Statistical hypothesis testing (Chi-square, ANOVA) to identify patterns and imbalances.
The Model Training phase followed the assignment requirement to:
 - Train a CNN from scratch – a custom convolutional neural network was designed with multiple Conv2D, MaxPooling, and Dense layers. While simpler than pre-trained architectures, it set a solid baseline.
 - Experiment with five pre-trained models using Transfer Learning:
   1. VGG16
   2. ResNet50
   3. MobileNet
   4. InceptionV3
   5. EfficientNetB0
   
- For each pre-trained model, ImageNet weights were loaded, the convolutional base was frozen initially, and new dense layers were added for classification into the target species. Models were compiled with the Adam optimizer, categorical cross-entropy loss, and accuracy as the evaluation metric.
- To further optimize performance, we conducted hyperparameter tuning on each architecture by experimenting with different learning rates, dense layer units, and dropout rates. In some cases, fine-tuning was performed by unfreezing selected higher convolutional layers and re-training with a lower learning rate.
- Model evaluation was based on:
 - Accuracy, Precision, Recall, F1-score
Confusion Matrices to visualize class-wise predictions
 - Training history plots (accuracy and loss) for monitoring learning behavior.
- Results demonstrated that while the custom CNN performed reasonably well, transfer learning models significantly outperformed it. Among them, MobileNet (example — replace with your highest) achieved the highest validation and test accuracy, making it the chosen model for deployment.
- Following evaluation, the best-performing model was saved in .keras format and integrated into a Streamlit web application. The app allows users to:
 - Upload a fish image (JPG/PNG)
 - Automatically preprocess the image
 - Predict the species and display it along with a confidence score
 - Show a real-time classification result through an intuitive interface

- Business Impact & Use Cases:
 - This classification system can help marine biologists, fishermen, and seafood distributors quickly identify fish species, reducing human error and improving operational efficiency. It can also support biodiversity studies and assist in monitoring fish populations.
 - Key Learnings & Skills Acquired:
  - Data preprocessing and augmentation for image models
  - Building CNNs from scratch
  - Applying Transfer Learning and fine-tuning
  - Model evaluation with detailed statistical and visual tools
  - Deployment using Streamlit and ngrok
  - Hands-on experience with TensorFlow/Keras and Python-based ML workflows
- Conclusion:
 - The project successfully delivered a full machine learning solution — from initial data exploration to deployed application. Systematic experimentation and evaluation allowed selection of an optimal architecture, demonstrating the effectiveness of transfer learning for domain-specific image classification tasks. The pipeline is scalable, allowing more classes or updated datasets to be incorporated with minimal changes.




# **GitHub Link -**

https://github.com/kshitiz562

# **Problem Statement**


- The identification of fish species from images is a critical challenge in fisheries management and biodiversity conservation. Manual classification is often slow and requires expert knowledge, highlighting the need for automated solutions.

- This project aims to automate fish species classification using deep learning and transfer learning techniques. We will develop and optimize various convolutional neural network (CNN) architectures, including VGG16, ResNet50, MobileNet, InceptionV3, and EfficientNetB0, on a labeled fish image dataset.

- Key challenges such as class imbalance and variable image quality will be addressed, and a robust training pipeline with data augmentation will be established. Furthermore, the final model will be integrated into a Streamlit web application for real-time predictions and confidence scores.

- By providing an efficient and user-friendly classification system, this project will enhance the automation of fish species identification and support large-scale monitoring in environmental and commercial applications.

# ***Let's Begin !***

## ***1. Know Your Data***

### Import Libraries

In [ ]:
# Install necessary libraries

!pip install numpy pandas matplotlib seaborn opencv-python pillow tensorflow scikit-learn optuna shap streamlit opencv-python pyngrok ngrok

In [ ]:
# General Utilities
import os
import random
import numpy as np
import pandas as pd

# Deep Learning - TensorFlow and Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers, callbacks
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.utils import to_categorical

# Pre-trained Model Architectures
from tensorflow.keras.applications import (
    VGG16,
    ResNet50,
    MobileNet,
    InceptionV3,
    EfficientNetB0
)
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg16_preprocess
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess
from tensorflow.keras.applications.mobilenet import preprocess_input as mobilenet_preprocess
from tensorflow.keras.applications.inception_v3 import preprocess_input as inception_preprocess
from tensorflow.keras.applications.efficientnet import preprocess_input as efficientnet_preprocess

# Model Evaluation
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score
)

# Data Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Model Serialization
import pickle

# Streamlit for Deployment
import streamlit as st

# Utilities
from PIL import Image

# Set random seeds for reproducibility
import tensorflow as tf
tf.random.set_seed(42)
np.random.seed(42)
random.seed(42)


### Dataset Loading

In [ ]:
# Load Dataset
# Mount Drive
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

# Load the zipfile
import os
import zipfile
zip_file_path = '/content/drive/MyDrive/Dataset.zip'
extract_path='/content/fish_images'
os.makedirs(extract_path, exist_ok=True)
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)
    print("Dataset has been extracted successfully.")


In [ ]:
import tensorflow as tf
import os

# Define the main dataset directory where you've extracted your images
dataset_path = '/content/fish_images/images.cv_jzk6llhf18tm3k0kyttxz/data'

# Define subdirectories for train, validation, and test datasets
train_dir = os.path.join(dataset_path, 'train')
val_dir = os.path.join(dataset_path, 'val')
test_dir = os.path.join(dataset_path, 'test')

# Set image size and batch size suitable for most transfer learning architectures
img_size = (224, 224)
batch_size = 32

# Create ImageDataGenerator instances with data augmentation for training and normalization for all
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    zoom_range=0.15,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

# Create dataset generators
train_dataset = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

val_dataset = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

test_dataset = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Check class labels mapping
print("Class indices:", train_dataset.class_indices)

### Create Images Dataset

In [ ]:
# Define the base directory where the fish images are stored
dataset_path = '/content/fish_images/images.cv_jzk6llhf18tm3k0kyttxz/data'

# Define subdirectories for train, validation, and test datasets
train_dir = os.path.join(dataset_path, 'train')
val_dir = os.path.join(dataset_path, 'val')
test_dir = os.path.join(dataset_path, 'test')

# Create empty lists to store the file paths and labels for each dataset
train_filepaths = []
train_labels = []
val_filepaths = []
val_labels = []
test_filepaths = []
test_labels = []

# Iterate through each class directory in the training set
for class_name in os.listdir(train_dir):
    class_dir = os.path.join(train_dir, class_name)
    if os.path.isdir(class_dir):
        for image_name in os.listdir(class_dir):
            image_path = os.path.join(class_dir, image_name)
            train_filepaths.append(image_path)
            train_labels.append(class_name)

# Iterate through each class directory in the validation set
for class_name in os.listdir(val_dir):
    class_dir = os.path.join(val_dir, class_name)
    if os.path.isdir(class_dir):
        for image_name in os.listdir(class_dir):
            image_path = os.path.join(class_dir, image_name)
            val_filepaths.append(image_path)
            val_labels.append(class_name)

# Iterate through each class directory in the test set
for class_name in os.listdir(test_dir):
    class_dir = os.path.join(test_dir, class_name)
    if os.path.isdir(class_dir):
        for image_name in os.listdir(class_dir):
            image_path = os.path.join(class_dir, image_name)
            test_filepaths.append(image_path)
            test_labels.append(class_name)

print(f"Number of training images found: {len(train_filepaths)}")
print(f"Number of validation images found: {len(val_filepaths)}")
print(f"Number of test images found: {len(test_filepaths)}")

In [ ]:
# Create DataFrames
train_df = pd.DataFrame({'filepath': train_filepaths, 'label': train_labels})
val_df = pd.DataFrame({'filepath': val_filepaths, 'label': val_labels})
test_df = pd.DataFrame({'filepath': test_filepaths, 'label': test_labels})

# Display the head of each DataFrame
print("Train DataFrame Head:")
display(train_df.head())

print("\nValidation DataFrame Head:")
display(val_df.head())

print("\nTest DataFrame Head:")
display(test_df.head())

In [ ]:
train_df.to_csv('/content/drive/MyDrive/my_data/train.csv', index=False)
val_df.to_csv('/content/drive/MyDrive/my_data/val.csv', index=False)
test_df.to_csv('/content/drive/MyDrive/my_data/test.csv', index=False)

### Dataset First View

In [ ]:
# Dataset First Look
train=pd.read_csv('/content/drive/MyDrive/my_data/train.csv')
val=pd.read_csv('/content/drive/MyDrive/my_data/val.csv')
test=pd.read_csv('/content/drive/MyDrive/my_data/test.csv')
print(train.head())
print(val.head())
print(test.head())

### Dataset Rows & Columns count

In [ ]:
# Dataset Rows & Columns count
print("Train Dataset Rows and Columns:", train.shape)
print("Validation Dataset Rows and Columns:", val.shape)
print("Test Dataset Rows and Columns:", test.shape)

### Dataset Information

In [ ]:
# Dataset Info
print("Train Dataset Information:")
print(train.info())

print("\nValidation Dataset Information:")
print(val.info())

print("\nTest Dataset Information:")
print(test.info())

#### Duplicate Values

In [ ]:
# Dataset Duplicate Value Count
print("Train Dataset Duplicate Values:")
print(train.duplicated().sum())

print("\nValidation Dataset Duplicate Values:")
print(val.duplicated().sum())

print("\nTest Dataset Duplicate Values:")
print(test.duplicated().sum())

#### Missing Values/Null Values

In [ ]:
# Missing Values/Null Values Count
print("Train Dataset Missing Values:")
print(train.isnull().sum())

print("\nValidation Dataset Missing Values:")
print(val.isnull().sum())

print("\nTest Dataset Missing Values:")
print(test.isnull().sum())

In [ ]:
# Visualizing the missing values
import seaborn as sns
import matplotlib.pyplot as plt

# Create a heatmap to visualize missing values
plt.figure(figsize=(10, 6))
sns.heatmap(train.isnull(), cbar=False, cmap='viridis')
plt.title('Train Missing Values Heatmap')
plt.show()
sns.heatmap(val.isnull(), cbar=False, cmap='viridis')
plt.title('Valid Missing Values Heatmap')
plt.show()
sns.heatmap(test.isnull(), cbar=False, cmap='viridis')
plt.title('Test Missing Values Heatmap')
plt.show()


### What did you know about your dataset?

- Prior to initiating the data analysis and model development processes, the foundational understanding of the dataset was derived from its source and accompanying description. This dataset is recognized as a compilation of images designated for multiclass fish classification. It is anticipated that the images will depict a variety of distinct fish species, with each image linked to an appropriate class label.

- It is expected that the dataset is structured into directories, with each directory potentially corresponding to a specific fish class. The images themselves are presumed to be standard photographic representations of fish. However, specific data attributes such as the total number of classes, the distribution of images across classes, image dimensions, and any potential data quality concerns remained unknown prior to the exploratory data analysis phase.


## ***2. Understanding Your Variables***

In [ ]:
# Dataset Columns
print("Train Dataset Columns:")
print(train.columns)

print("\nValidation Dataset Columns:")
print(val.columns)

print("\nTest Dataset Columns:")
print(test.columns)

In [ ]:
# Dataset Describe
print("Train Dataset Describe:")
print(train.describe())

print("\nValidation Dataset Describe:")
print(val.describe())

print("\nTest Dataset Describe:")
print(test.describe())

### Variables Description

- Upon converting the dataset into DataFrames labeled as train_df, val_df, and test_df, each dataset comprises the following variables (columns):

**Variable Name** | **Data Type** | **Description**  
--- | --- | ---  
filepath | string | The complete file path to the image stored on disk. This is utilized by the model loading pipeline (ImageDataGenerator) to read and preprocess the corresponding image.  
label | string | The target class name (fish species) associated with the image. This label is derived from the folder name in which the image resides and serves as the dependent variable for the classification task.

- In addition, several temporary variables created during the analysis, which are not included in the final training dataset, include:

**Variable Name** | **Data Type** | **Description**  
--- | --- | ---  
filepath_length | int | The length (number of characters) of the image's file path, employed solely for exploratory purposes to identify any unusually long paths or storage inconsistencies.  
avg_color | list[float] | The average RGB values calculated for the image, utilized in Exploratory Data Analysis (EDA) to examine potential color distribution differences across classes.  
brightness | float | The average grayscale intensity value of the image, intended for analyzing brightness distribution.

- The label field is categorical, containing values that correspond to the names of fish species. The filepath field is unique for each image and facilitates the location and loading of images for training or testing purposes. During the preprocessing phase for modeling, the label variable is one-hot encoded into numerical vectors, while the filepath variable is employed to provide images to both the Convolutional Neural Network (CNN) and transfer learning models.

### Check Unique Values for each variable.

In [ ]:
# Check Unique Values for each variable.
print("Unique Values in Train Dataset:")
print(train.nunique())

print("\nUnique Values in Validation Dataset:")
print(val.nunique())

print("\nUnique Values in Test Dataset:")
print(test.nunique())

## 3. ***Data Wrangling***

### Data Wrangling Code

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import cv2

# 1. Class Balance Visualization
plt.figure(figsize=(10,4))
sns.countplot(y=train['label'], order=train['label'].value_counts().index)
plt.title('Number of Images per Fish Class (Train Set)')
plt.xlabel('Count')
plt.ylabel('Fish Species')
plt.show()

# 2. Sample Image Visualization
def show_samples(df, class_col='label', filepath_col='filepath', samples_per_class=3):
    classes = df[class_col].unique()
    n_classes = len(classes)
    plt.figure(figsize=(samples_per_class*3, n_classes*3))
    for idx, fish_class in enumerate(classes):
        imgs = df[df[class_col]==fish_class].sample(samples_per_class, random_state=42)[filepath_col].tolist()
        for i, img_path in enumerate(imgs):
            plt.subplot(n_classes, samples_per_class, idx*samples_per_class+i+1)
            img = cv2.imread(img_path)
            if img is not None:
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                plt.imshow(img)
                plt.axis('off')
                plt.title(fish_class)
    plt.tight_layout()
    plt.show()

show_samples(train, 'label', 'filepath', samples_per_class=3)

# 3. Image Dimension Consistency Check (Check for corrupt/irregular images)
bad_images = []
for i, row in train.sample(50, random_state=1).iterrows():  # Check a random subset
    try:
        img = cv2.imread(row['filepath'])
        assert img is not None
        assert img.shape[-1] == 3
    except Exception as e:
        print(f"Bad image: {row['filepath']} Reason: {e}")
        bad_images.append(row['filepath'])
if not bad_images:
    print("No corrupt or unexpected images found in sample.")

# 4. Save class indices for label decoding (and optionally map to readable names)
import json
class_indices = train_dataset.class_indices  # Use from earlier Keras flow_from_directory output
with open('/content/drive/MyDrive/my_data/class_indices.json', 'w') as f:
    json.dump(class_indices, f)
print("Class indices mapping saved.")


### What all manipulations have you done and insights you found?

**Manipulations Conducted:**

- The dataset was extracted from Google Drive, and the folder structure was carefully verified (i.e., train/, val/, test/). Subsequently, training, validation, and test data frames (train_df, val_df, test_df) were generated, which included file paths and corresponding labels, and saved as CSV files. A thorough examination confirmed the absence of missing values and duplicates. Furthermore, it was validated that all images are readable in RGB format with consistent dimensions, having been resized to (224, 224) pixels.

- ImageDataGenerator pipelines with augmentation for the training dataset and rescaling for validation and test datasets were established. Class distributions were plotted, sample images were displayed, and an analysis of image sizes, aspect ratios, brightness, and average colors was conducted. A mapping of class indices was saved for use during prediction tasks. Statistical tests, including Chi-square and ANOVA, were performed to assess balance and variance in brightness.

**Insights Obtained:**

 - The dataset is clean, exhibiting no missing, duplicate, or corrupt images. It encompasses [X] species; while some class imbalance is observed, augmentation techniques effectively mitigate this issue. Notable variations in brightness and color across species may enhance classification accuracy. The high intra-class variation and inter-class similarity provide substantial justification for employing deep transfer learning models. Consequently, the final dataset is deemed ready and suitable for comprehensive deep learning training.




## ***4. Data Vizualization, Storytelling & Experimenting with charts : Understand the relationships between variables***

#### Chart - 1: Class Distribution in Train Set

In [ ]:
# Chart - 1: Class Distribution in Train Set
plt.figure(figsize=(10, 6))
sns.countplot(y=train['label'], order=train['label'].value_counts().index, palette='viridis')
plt.title('Number of Images per Fish Class (Train Set)', fontsize=14)
plt.xlabel('Count', fontsize=12)
plt.ylabel('Fish Species', fontsize=12)
plt.show()

##### 1. Why did you pick the specific chart?

- To visualize the count of images for each fish class in the training set and identify potential class imbalance.

##### 2. What is/are the insight(s) found from the chart?

- The chart shows that the distribution of images across different fish classes is uneven, indicating class imbalance.

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

-  Yes, understanding class imbalance is crucial to apply appropriate techniques (like weighting or augmentation) to ensure the model performs well on all fish types, leading to more reliable identification in a business context.

#### Chart - 2: Sample Image Visualization

In [ ]:
# Chart - 2: Sample Image Visualization
def show_samples(df, class_col='label', filepath_col='filepath', samples_per_class=3):
    classes = df[class_col].unique()
    n_classes = len(classes)
    plt.figure(figsize=(samples_per_class*3, n_classes*3))
    for idx, fish_class in enumerate(classes):
        class_samples = df[df[class_col]==fish_class]
        num_samples_to_show = min(samples_per_class, len(class_samples))
        if num_samples_to_show > 0:
            imgs = class_samples.sample(num_samples_to_show, random_state=42)[filepath_col].tolist()
            for i, img_path in enumerate(imgs):
                plt.subplot(n_classes, samples_per_class, idx*samples_per_class+i+1)
                img = cv2.imread(img_path)
                if img is not None:
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    plt.imshow(img)
                    plt.axis('off')
                    plt.title(fish_class)
    plt.tight_layout()
    plt.show()

show_samples(train, 'label', 'filepath', samples_per_class=3)

##### 1. Why did you pick the specific chart?

- To visually inspect representative images from each fish class and understand the visual characteristics of the data.

##### 2. What is/are the insight(s) found from the chart?

-  Provides a visual sense of image quality, variations within a class, and visual differences between various fish species.

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

-  Yes, visually inspecting data helps confirm it aligns with expectations and can reveal data quality issues that need addressing for accurate classification.

#### Chart - 3: Image Dimension Consistency Check

In [ ]:
# Chart - 3: Image Dimension Consistency Check
from PIL import Image
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns

def check_image_dimensions(dataframe, filepath_col='filepath', sample_size=500):
    dimensions = []
    sampled_df = dataframe.sample(min(sample_size, len(dataframe)), random_state=42)
    print(f"Checking dimensions of {len(sampled_df)} images...")
    for index, row in sampled_df.iterrows():
        try:
            img = Image.open(row[filepath_col])
            dimensions.append(img.size)
        except Exception as e:
            print(f"Could not open image {row[filepath_col]}: {e}")
            dimensions.append(None)
    return dimensions

train_dimensions = check_image_dimensions(train_df)

valid_train_dimensions = [d for d in train_dimensions if d is not None]

dimension_counts = Counter(valid_train_dimensions)

dimension_df = pd.DataFrame.from_records(list(dimension_counts.items()), columns=['Dimension', 'Count'])
dimension_df['Dimension_Str'] = dimension_df['Dimension'].apply(str)

# Plot dimension counts
plt.figure(figsize=(10, 6))
sns.barplot(x='Dimension_Str', y='Count', data=dimension_df, palette='viridis')
plt.title('Distribution of Image Dimensions (Sample)', fontsize=14)
plt.xlabel('Image Dimension (Width, Height)', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

if len(dimension_counts) > 1:
    print("\nWarning: Multiple image dimensions found in the sample. Consider resizing images during preprocessing.")
elif len(dimension_counts) == 1:
     print("\nImages in the sample have a consistent dimension.")
else:
    print("\nNo valid image dimensions found in the sample.")

##### 1. Why did you pick the specific chart?

-  To check if all images have consistent dimensions and visualize the distribution of sizes in the dataset.

##### 2. What is/are the insight(s) found from the chart?

- Showed whether image dimensions are uniform or if variation exists, indicating the need for resizing during preprocessing.

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

- Yes, consistent image dimensions are essential for model input and processing, preventing errors and ensuring the model trains effectively on standardized data.

#### Chart - 4: Class Distribution across Train, Validation, and Test Sets

In [ ]:
# Chart - 4: Class Distribution across Train, Validation, and Test Sets
fig, axes = plt.subplots(1, 3, figsize=(20, 6))
fig.suptitle('Class Distribution Across Datasets', fontsize=16)

# Train set class distribution
sns.countplot(y=train_df['label'], order=train_df['label'].value_counts().index, palette='viridis', ax=axes[0])
axes[0].set_title('Train Set Class Distribution')
axes[0].set_xlabel('Count')
axes[0].set_ylabel('Fish Species')

# Validation set class distribution
sns.countplot(y=val_df['label'], order=val_df['label'].value_counts().index, palette='viridis', ax=axes[1])
axes[1].set_title('Validation Set Class Distribution')
axes[1].set_xlabel('Count')
axes[1].set_ylabel('')

# Test set class distribution
sns.countplot(y=test_df['label'], order=test_df['label'].value_counts().index, palette='viridis', ax=axes[2])
axes[2].set_title('Test Set Class Distribution')
axes[2].set_xlabel('Count')
axes[2].set_ylabel('')

plt.tight_layout(rect=[0, 0.03, 1, 0.95])
plt.show()

##### 1. Why did you pick the specific chart?

- To compare the distribution of fish classes across the training, validation, and test datasets.

##### 2. What is/are the insight(s) found from the chart?

- Verified that the class distribution is relatively similar across the different data splits

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

-  Yes, similar distributions across splits ensure that model evaluation results on the validation and test sets are reliable and representative of performance on unseen data.

#### Chart - 5: Distribution of Filepath Lengths

In [ ]:
# Chart - 5: Distribution of Filepath Lengths
train_df['filepath_length'] = train_df['filepath'].apply(len)
val_df['filepath_length'] = val_df['filepath'].apply(len)
test_df['filepath_length'] = test_df['filepath'].apply(len)

plt.figure(figsize=(10, 6))
sns.histplot(train_df['filepath_length'], kde=True, color='skyblue', label='Train')
sns.histplot(val_df['filepath_length'], kde=True, color='lightcoral', label='Validation')
sns.histplot(test_df['filepath_length'], kde=True, color='lightgreen', label='Test')
plt.title('Distribution of Filepath Lengths', fontsize=14)
plt.xlabel('Filepath Length', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.legend()
plt.show()

train_df = train_df.drop('filepath_length', axis=1)
val_df = val_df.drop('filepath_length', axis=1)
test_df = test_df.drop('filepath_length', axis=1)

##### 1. Why did you pick the specific chart?

- To visualize a basic characteristic of the dataset's file paths.

##### 2. What is/are the insight(s) found from the chart?

-  Showed the range and frequency of lengths for the image file paths.

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

-  Limited direct impact on classification performance, but relevant for data management or system design considerations.

#### Chart - 6: Average Color Distribution

In [ ]:
# Chart - 6: Average Color Distribution
import cv2
import numpy as np

def get_average_color(image_path, size=(100, 100)):
    try:
        img = cv2.imread(image_path)
        if img is not None:
            img = cv2.resize(img, size)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            return np.mean(img, axis=(0, 1))
        else:
            return None
    except Exception as e:
        print(f"Could not process image {image_path}: {e}")
        return None

sample_size = 500
sampled_train_df = train_df.sample(min(sample_size, len(train_df)), random_state=42).copy()
sampled_train_df['avg_color'] = sampled_train_df['filepath'].apply(get_average_color)

sampled_train_df.dropna(subset=['avg_color'], inplace=True)

colors_df = pd.DataFrame(sampled_train_df['avg_color'].tolist(), columns=['avg_red', 'avg_green', 'avg_blue'])
colors_df['label'] = sampled_train_df['label'].values

plt.figure(figsize=(12, 7))
sns.boxplot(x='label', y='avg_red', data=colors_df, palette='Reds')
plt.title('Average Red Channel Value Distribution by Class (Sample)', fontsize=14)
plt.xlabel('Fish Species', fontsize=12)
plt.ylabel('Average Red Value', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

plt.figure(figsize=(12, 7))
sns.boxplot(x='label', y='avg_green', data=colors_df, palette='Greens')
plt.title('Average Green Channel Value Distribution by Class (Sample)', fontsize=14)
plt.xlabel('Fish Species', fontsize=12)
plt.ylabel('Average Green Value', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

plt.figure(figsize=(12, 7))
sns.boxplot(x='label', y='avg_blue', data=colors_df, palette='Blues')
plt.title('Average Blue Channel Value Distribution by Class (Sample)', fontsize=14)
plt.xlabel('Fish Species', fontsize=12)
plt.ylabel('Average Blue Value', fontsize=12)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

##### 1. Why did you pick the specific chart?

-  To explore the average color characteristics of images within and across different fish classes.

##### 2. What is/are the insight(s) found from the chart?

Answer Here

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

Answer Here

#### Chart - 7:  Image Brightness Distribution

In [ ]:
# Chart - 7: Image Brightness Distribution
import cv2
import numpy as np
from PIL import Image

def get_image_brightness(image_path, size=(100, 100)):
    try:
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        if img is not None:
            img = cv2.resize(img, size)
            return np.mean(img)
        else:
            return None
    except Exception as e:
        print(f"Could not process image {image_path}: {e}")
        return None

sample_size = 500
sampled_train_df = train_df.sample(min(sample_size, len(train_df)), random_state=42).copy()
sampled_train_df['brightness'] = sampled_train_df['filepath'].apply(get_image_brightness)

sampled_train_df.dropna(subset=['brightness'], inplace=True)

plt.figure(figsize=(10, 6))
sns.histplot(sampled_train_df['brightness'], kde=True, color='orange')
plt.title('Distribution of Image Brightness (Sample)', fontsize=14)
plt.xlabel('Average Pixel Intensity (Brightness)', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.show()

##### 1. Why did you pick the specific chart?

- To visualize the distribution of overall image brightness in a sample of the dataset.

##### 2. What is/are the insight(s) found from the chart?

- Showed the range of lighting conditions present in the images.

##### 3. Will the gained insights help creating a positive business impact?
Are there any insights that lead to negative growth? Justify with specific reason.

-  Yes, understanding brightness variation can inform the need for data augmentation techniques like brightness adjustments to improve model robustness to different lighting conditions in real-world scenarios.

## ***5. Hypothesis Testing***

### Based on your chart experiments, define three hypothetical statements from the dataset. In the next three questions, perform hypothesis testing to obtain final conclusion about the statements through your code and statistical testing.

- Based on the insights gained from the data visualization and exploration in Section 4, we define the following three hypothetical statements about the fish image dataset:

 1.  **Hypothetical Statement 1:** The distribution of image counts across different fish classes is significantly uneven in the training dataset.
 2.  **Hypothetical Statement 2:** The average brightness of images differs significantly between at least two different fish classes.
 3.  **Hypothetical Statement 3:** The image dimensions are consistent across the entire dataset.

### Hypothetical Statement - 1

#### 1. State Your research hypothesis as a null hypothesis and alternate hypothesis.

- Null Hypothesis (H0): The image counts are equally distributed across all fish classes in the training dataset.
- Alternate Hypothesis (H1): The image counts are not equally distributed across all fish classes in the training dataset.

#### 2. Perform an appropriate statistical test.

In [ ]:
# Perform Statistical Test to obtain P-Value
from scipy.stats import chisquare
import pandas as pd

observed_counts = train_df['label'].value_counts()

num_classes = len(observed_counts)
total_images = observed_counts.sum()

expected_counts = total_images / num_classes

# Perform the Chi-Squared Goodness of Fit Test
chi2_statistic, p_value = chisquare(f_obs=observed_counts, f_exp=expected_counts)

print(f"Observed Counts:\n{observed_counts}")
print(f"\nNumber of Classes: {num_classes}")
print(f"Total Images: {total_images}")
print(f"Expected Count per Class (under H0): {expected_counts:.2f}")
print(f"\nChi-Squared Statistic: {chi2_statistic:.4f}")
print(f"P-Value: {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("\nResult: Reject the null hypothesis. There is significant evidence that the distribution of image counts across classes is uneven.")
else:
    print("\nResult: Fail to reject the null hypothesis. There is not enough evidence to say that the distribution of image counts across classes is uneven.")

##### Which statistical test have you done to obtain P-Value?

- Chi-Squared Goodness of Fit Test.

##### Why did you choose the specific statistical test?

- This test was chosen because it is suitable for comparing observed frequencies (image counts per class) to expected frequencies (equal distribution) in categorical data to determine if the difference is statistically significant.

### Hypothetical Statement - 2

#### 1. State Your research hypothesis as a null hypothesis and alternate hypothesis.

- Null Hypothesis (H0): The average brightness is the same for all fish classes in the training dataset.
- Alternate Hypothesis (H1): The average brightness is different for at least two fish classes in the training dataset.

#### 2. Perform an appropriate statistical test.

In [ ]:
# Perform Statistical Test to obtain P-Value
from scipy import stats

# Prepare data for ANOVA
brightness_by_class = [sampled_train_df['brightness'][sampled_train_df['label'] == cls].dropna().tolist()
                       for cls in sampled_train_df['label'].unique()]

# Perform one-way ANOVA test
valid_groups = [group for group in brightness_by_class if len(group) > 1]

if len(valid_groups) >= 2:
    f_statistic, p_value = stats.f_oneway(*valid_groups)

    print(f"ANOVA F-Statistic: {f_statistic:.4f}")
    print(f"ANOVA P-Value: {p_value:.4f}")

    alpha = 0.05
    if p_value < alpha:
        print("\nResult: Reject the null hypothesis. There is significant evidence that the average brightness differs between at least two fish classes.")
    else:
        print("\nResult: Fail to reject the null hypothesis. There is not enough evidence to say that the average brightness differs between fish classes.")
else:
    print("ANOVA test requires at least two groups with more than one sample each.")

##### Which statistical test have you done to obtain P-Value?

- One-Way ANOVA (Analysis of Variance) Test.

##### Why did you choose the specific statistical test?

- ANOVA is the appropriate test for comparing the means of three or more independent groups (average brightness for each fish class) to assess if at least one group mean is statistically different from the others.

### Hypothetical Statement - 3

#### 1. State Your research hypothesis as a null hypothesis and alternate hypothesis.

- Null Hypothesis (H0): All images in the dataset have the same dimensions.
- Alternate Hypothesis (H1): Not all images in the dataset have the same dimensions (i.e., there is inconsistency in image dimensions).

#### 2. Perform an appropriate statistical test.

In [ ]:
# Perform Statistical Test to obtain P-Value
from PIL import Image
from collections import Counter

def check_image_dimensions_for_hypothesis(dataframe, filepath_col='filepath', sample_size=500):
    dimensions = []
    sampled_df = dataframe.sample(min(sample_size, len(dataframe)), random_state=42)
    print(f"Checking dimensions of {len(sampled_df)} images for hypothesis testing...")
    for index, row in sampled_df.iterrows():
        try:
            img = Image.open(row[filepath_col])
            dimensions.append(img.size)
        except Exception as e:
            pass
    return dimensions

train_dimensions_sample = check_image_dimensions_for_hypothesis(train_df)

val_dimensions_sample = check_image_dimensions_for_hypothesis(val_df)

test_dimensions_sample = check_image_dimensions_for_hypothesis(test_df)


all_sampled_dimensions = train_dimensions_sample + val_dimensions_sample + test_dimensions_sample
unique_sampled_dimensions = set(all_sampled_dimensions)

print("\nUnique dimensions found across sampled images:", unique_sampled_dimensions)

if len(unique_sampled_dimensions) == 1 and unique_sampled_dimensions != {None}:
    print("\nResult: Based on the sample, we fail to reject the null hypothesis. The image dimensions appear to be consistent across the sampled dataset.")
elif len(unique_sampled_dimensions) > 1:
    print("\nResult: Based on the sample, we reject the null hypothesis. The image dimensions are not consistent across the sampled dataset.")
else:
    print("\nCould not determine consistency from the sampled images.")

##### Which statistical test have you done to obtain P-Value?

- Direct programmatic check and analysis of unique dimensions found in sampled images.

##### Why did you choose the specific statistical test?

- A traditional p-value based test is not standard for this specific check. The direct analysis was chosen as it directly verifies the condition of equality of dimensions based on observed data.

## ***6. ML Model Implementation***

### ML Model - 1

In [ ]:
# ML Model - 1 Implementation (CNN from Scratch)

# CNN model architecture
model_cnn = Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_size[0], img_size[1], 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(train_dataset.num_classes, activation='softmax')
])

# Compile the model
model_cnn.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

# Display model summary
model_cnn.summary()

# Train the model
epochs = 5
history_cnn = model_cnn.fit(
    train_dataset,
    steps_per_epoch=train_dataset.samples // batch_size,
    epochs=epochs,
    validation_data=val_dataset,
    validation_steps=val_dataset.samples // batch_size
)

# Save the trained model
model_cnn.save('/content/drive/MyDrive/my_models/cnn_model.keras')
print("\nCNN model saved successfully.")

#### 1. Explain the ML Model used and it's performance using Evaluation metric Score Chart.

In [ ]:
# Visualizing evaluation Metric Score chart

# Load the saved model
loaded_model_cnn = load_model('/content/drive/MyDrive/my_models/cnn_model.keras')
print("CNN model loaded successfully.")

print("\nEvaluation of CNN model on the test dataset")
loss_cnn, accuracy_cnn = loaded_model_cnn.evaluate(test_dataset)

print(f"\nTest Loss: {loss_cnn:.4f}")
print(f"Test Accuracy: {accuracy_cnn:.4f}")

test_predictions_cnn = loaded_model_cnn.predict(test_dataset, verbose=0)
test_predicted_labels_cnn = np.argmax(test_predictions_cnn, axis=1)

# Get true labels
test_true_labels = test_dataset.classes
class_names = list(train_dataset.class_indices.keys())

# Generate Classification Report
print("\nClassification Report (CNN Model):")
print(classification_report(test_true_labels, test_predicted_labels_cnn, target_names=class_names))

# Generate Confusion Matrix
print("\nConfusion Matrix (CNN Model):")
cm_cnn = confusion_matrix(test_true_labels, test_predicted_labels_cnn)

# Visualize Confusion Matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm_cnn, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix (CNN Model)')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# Visualize Training History
if 'history_cnn' in locals():
    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    plt.plot(history_cnn.history['accuracy'])
    plt.plot(history_cnn.history['val_accuracy'])
    plt.title('Model Accuracy (CNN)')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    plt.subplot(1, 2, 2)
    plt.plot(history_cnn.history['loss'])
    plt.plot(history_cnn.history['val_loss'])
    plt.title('Model Loss (CNN)')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    plt.tight_layout()
    plt.show()
else:
    print("\nTraining history object 'history_cnn' not found. Cannot plot training history.")

#### 2. Cross- Validation & Hyperparameter Tuning



In [ ]:
# ML Model - 1 Hyperparameter Tuning (CNN from Scratch - Simplified Example)

learning_rates = [0.001, 0.0005]
dense_units = [64, 128]
dropout_rates = [0.4, 0.5]

best_accuracy = 0
best_hyperparameters = {}

for lr in learning_rates:
    for units in dense_units:
        for dropout in dropout_rates:
            print(f"\nAttempting training with LR: {lr}, Dense Units: {units}, Dropout: {dropout}")

            # CNN model with current hyperparameters
            model_tuned_cnn = Sequential([
                keras.Input(shape=(img_size[0], img_size[1], 3)),
                layers.Conv2D(32, (3, 3), activation='relu'),
                layers.MaxPooling2D((2, 2)),
                layers.Conv2D(64, (3, 3), activation='relu'),
                layers.MaxPooling2D((2, 2)),
                layers.Conv2D(128, (3, 3), activation='relu'),
                layers.MaxPooling2D((2, 2)),
                layers.Flatten(),
                layers.Dense(units, activation='relu'),
                layers.Dropout(dropout),
                layers.Dense(train_dataset.num_classes, activation='softmax')
            ])

            # Compile the model
            model_tuned_cnn.compile(optimizer=optimizers.Adam(learning_rate=lr),
                                    loss='categorical_crossentropy',
                                    metrics=['accuracy'])

            epochs_tune = 1
            history_tuned_cnn = model_tuned_cnn.fit(
                train_dataset,
                steps_per_epoch=train_dataset.samples // batch_size,
                epochs=epochs_tune,
                validation_data=val_dataset,
                validation_steps=val_dataset.samples // batch_size,
                verbose=0
            )

            # Evaluate on validation set
            val_loss, val_accuracy = model_tuned_cnn.evaluate(val_dataset, verbose=0)
            print(f"Validation Accuracy for current trial: {val_accuracy:.4f}")

            if val_accuracy > best_accuracy:
                best_accuracy = val_accuracy
                best_hyperparameters = {'learning_rate': lr, 'dense_units': units, 'dropout_rate': dropout}
                print("New best hyperparameters found!")

print("\nHyperparameter tuning complete (Simplified Example).")
print("Best Validation Accuracy:", best_accuracy)
print("Best Hyperparameters:", best_hyperparameters)

##### Which hyperparameter optimization technique have you used and why?

-  We used a simplified manual grid search approach for hyperparameter tuning. This basic technique involves iterating through a small, predefined set of hyperparameter values (Learning Rate, Dense Units, Dropout Rate) to demonstrate the concept of exploring the hyperparameter space.

##### Have you seen any improvement? Note down the improvement with updates Evaluation metric Score Chart.

- Based on the limited trials performed before stopping early, we observed potential improvement. The highest validation accuracy achieved during this simplified tuning was [Insert Best Validation Accuracy from Output of cell Dy61ujd6fxKe] with the hyperparameters: [Insert Best Hyperparameters from Output of cell Dy61ujd6fxKe]. A full, comprehensive tuning process would be needed to confirm significant improvement and compare it to the initial model's test accuracy.

### ML Model - 2

In [ ]:
# ML Model - 2 Implementation (Transfer Learning with VGG16)

# Load the pre-trained VGG16 model
base_model_vgg16 = VGG16(weights='imagenet', include_top=False, input_shape=(img_size[0], img_size[1], 3))

for layer in base_model_vgg16.layers:
    layer.trainable = False

model_vgg16 = Sequential([
    base_model_vgg16,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(train_dataset.num_classes, activation='softmax')
])

# Compile the model
model_vgg16.compile(optimizer=optimizers.Adam(learning_rate=0.001),
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])

# Display model summary
model_vgg16.summary()

# Train the model
epochs_vgg16 = 5
history_vgg16 = model_vgg16.fit(
    train_dataset,
    steps_per_epoch=train_dataset.samples // batch_size,
    epochs=epochs_vgg16,
    validation_data=val_dataset,
    validation_steps=val_dataset.samples // batch_size
)

# Save the trained model
model_vgg16.save('/content/drive/MyDrive/my_models/vgg16_model.keras')
print("\nVGG16 model saved successfully.")

#### 1. Explain the ML Model used and it's performance using Evaluation metric Score Chart.

In [ ]:
# Visualizing evaluation Metric Score chart
# Load the saved VGG16 model
loaded_model_vgg16 = load_model('/content/drive/MyDrive/my_models/vgg16_model.keras')
print("VGG16 model loaded successfully.")

# Evaluate the loaded model on the test dataset
loss_vgg16, accuracy_vgg16 = loaded_model_vgg16.evaluate(test_dataset)

print(f"\nTest Loss: {loss_vgg16:.4f}")
print(f"Test Accuracy: {accuracy_vgg16:.4f}")

test_predictions_vgg16 = loaded_model_vgg16.predict(test_dataset, verbose=0)
test_predicted_labels_vgg16 = np.argmax(test_predictions_vgg16, axis=1)

test_true_labels = test_dataset.classes

# Get class names
class_names = list(train_dataset.class_indices.keys())

# Generate Classification Report
print("\nClassification Report (VGG16 Model):")
print(classification_report(test_true_labels, test_predicted_labels_vgg16, target_names=class_names))

# Generate Confusion Matrix
print("\nConfusion Matrix (VGG16 Model):")
cm_vgg16 = confusion_matrix(test_true_labels, test_predicted_labels_vgg16)

# Visualize Confusion Matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm_vgg16, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix (VGG16 Model)')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# Visualize Training History
if 'history_vgg16' in locals():
    plt.figure(figsize=(12, 6))

    # Plot training & validation accuracy values
    plt.subplot(1, 2, 1)
    plt.plot(history_vgg16.history['accuracy'])
    plt.plot(history_vgg16.history['val_accuracy'])
    plt.title('Model Accuracy (VGG16)')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    # Plot training & validation loss values
    plt.subplot(1, 2, 2)
    plt.plot(history_vgg16.history['loss'])
    plt.plot(history_vgg16.history['val_loss'])
    plt.title('Model Loss (VGG16)')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    plt.tight_layout()
    plt.show()
else:
    print("\nTraining history object 'history_vgg16' not found. Cannot plot training history.")

#### 2. Cross- Validation & Hyperparameter Tuning

In [ ]:
# ML Model - 2 Hyperparameter Tuning (Transfer Learning with VGG16)

# Load the pre-trained VGG16 base model
base_model_vgg16_tune = VGG16(weights='imagenet', include_top=False, input_shape=(img_size[0], img_size[1], 3))

for layer in base_model_vgg16_tune.layers:
    layer.trainable = False

learning_rates = [0.001, 0.0005]
dense_units = [128, 256]
dropout_rates = [0.4, 0.5]

best_accuracy_vgg16 = 0
best_hyperparameters_vgg16 = {}

stop_tuning_early = False

for lr in learning_rates:
    if stop_tuning_early:
        break
    for units in dense_units:
        if stop_tuning_early:
            break
        for dropout in dropout_rates:
            if stop_tuning_early:
                break

            print(f"\nAttempting training with LR: {lr}, Dense Units: {units}, Dropout: {dropout}")

            model_tuned_vgg16 = Sequential([
                base_model_vgg16_tune,
                layers.Flatten(),
                layers.Dense(units, activation='relu'),
                layers.Dropout(dropout),
                layers.Dense(train_dataset.num_classes, activation='softmax')
            ])

            # Compile the model
            model_tuned_vgg16.compile(optimizer=optimizers.Adam(learning_rate=lr),
                                      loss='categorical_crossentropy',
                                      metrics=['accuracy'])

            # Train the model
            epochs_tune = 1
            history_tuned_vgg16 = model_tuned_vgg16.fit(
                train_dataset,
                steps_per_epoch=train_dataset.samples // batch_size,
                epochs=epochs_tune,
                validation_data=val_dataset,
                validation_steps=val_dataset.samples // batch_size,
                verbose=0
            )

            # Evaluate on validation set
            val_loss, val_accuracy = model_tuned_vgg16.evaluate(val_dataset, verbose=0)
            print(f"Validation Accuracy for current trial: {val_accuracy:.4f}")

            if val_accuracy > best_accuracy_vgg16:
                best_accuracy_vgg16 = val_accuracy
                best_hyperparameters_vgg16 = {'learning_rate': lr, 'dense_units': units, 'dropout_rate': dropout}
                print("New best hyperparameters found!")
                stop_tuning_early = True

print("\nHyperparameter tuning complete (Simplified Example).")
print("Best Validation Accuracy:", best_accuracy_vgg16)
print("Best Hyperparameters:", best_hyperparameters_vgg16)

##### Which hyperparameter optimization technique have you used and why?

-  A simplified manual grid search was applied to the hyperparameters of the newly added layers on top of the pre-trained VGG16 base (Learning Rate, Dense Units, Dropout Rate). This method helps demonstrate tuning the classification head in a transfer learning setup.

##### Have you seen any improvement? Note down the improvement with updates Evaluation metric Score Chart.

- From the limited tuning trials, a validation accuracy of [Insert Best Validation Accuracy Here] was achieved with hyperparameters: [Insert Best Hyperparameters Here]. This suggests that tuning the top layers can influence performance, but a complete search is required for optimal results.

### ML Model - 3

In [ ]:
# ML Model - 3 Implementation (Transfer Learning with ResNet50)

# Load the pre-trained ResNet50 model
base_model_resnet50 = ResNet50(weights='imagenet', include_top=False, input_shape=(img_size[0], img_size[1], 3))

for layer in base_model_resnet50.layers:
    layer.trainable = False

model_resnet50 = Sequential([
    base_model_resnet50,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(train_dataset.num_classes, activation='softmax')
])

# Compile the model
model_resnet50.compile(optimizer=optimizers.Adam(learning_rate=0.001),
                       loss='categorical_crossentropy',
                       metrics=['accuracy'])

# Display model summary
model_resnet50.summary()

# Train the model
epochs_resnet50 = 5
history_resnet50 = model_resnet50.fit(
    train_dataset,
    steps_per_epoch=train_dataset.samples // batch_size,
    epochs=epochs_resnet50,
    validation_data=val_dataset,
    validation_steps=val_dataset.samples // batch_size
)

# Save the trained model
model_resnet50.save('/content/drive/MyDrive/my_models/resnet50_model.keras')
print("\nResNet50 model saved successfully.")

#### 1. Explain the ML Model used and it's performance using Evaluation metric Score Chart.

In [ ]:
# Visualizing evaluation Metric Score chart

# Load the saved ResNet50 model
loaded_model_resnet50 = load_model('/content/drive/MyDrive/my_models/resnet50_model.keras')
print("ResNet50 model loaded successfully.")

# Evaluate the loaded model on the test dataset
loss_resnet50, accuracy_resnet50 = loaded_model_resnet50.evaluate(test_dataset)

print(f"\nTest Loss: {loss_resnet50:.4f}")
print(f"Test Accuracy: {accuracy_resnet50:.4f}")

test_predictions_resnet50 = loaded_model_resnet50.predict(test_dataset, verbose=0)
test_predicted_labels_resnet50 = np.argmax(test_predictions_resnet50, axis=1)

# Get true labels
test_true_labels = test_dataset.classes

# Get class names
class_names = list(train_dataset.class_indices.keys())

# Generate Classification Report
print("\nClassification Report (ResNet50 Model):")
print(classification_report(test_true_labels, test_predicted_labels_resnet50, target_names=class_names))

# Generate Confusion Matrix
print("\nConfusion Matrix (ResNet50 Model):")
cm_resnet50 = confusion_matrix(test_true_labels, test_predicted_labels_resnet50)

# Visualize Confusion Matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm_resnet50, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix (ResNet50 Model)')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# Visualize Training History
if 'history_resnet50' in locals():
    plt.figure(figsize=(12, 6))

    # Plot training & validation accuracy values
    plt.subplot(1, 2, 1)
    plt.plot(history_resnet50.history['accuracy'])
    plt.plot(history_resnet50.history['val_accuracy'])
    plt.title('Model Accuracy (ResNet50)')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    # Plot training & validation loss values
    plt.subplot(1, 2, 2)
    plt.plot(history_resnet50.history['loss'])
    plt.plot(history_resnet50.history['val_loss'])
    plt.title('Model Loss (ResNet50)')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    plt.tight_layout()
    plt.show()
else:
    print("\nTraining history object 'history_resnet50' not found. Cannot plot training history.")

#### 2. Cross- Validation & Hyperparameter Tuning

In [ ]:
# ML Model - 3 Hyperparameter Tuning (Transfer Learning with ResNet50 - Simplified Example)

# Load the pre-trained ResNet50 base model
base_model_resnet50_tune = ResNet50(weights='imagenet', include_top=False, input_shape=(img_size[0], img_size[1], 3))

for layer in base_model_resnet50_tune.layers:
    layer.trainable = False

learning_rates = [0.001, 0.0005]
dense_units = [128, 256]
dropout_rates = [0.4, 0.5]

best_accuracy_resnet50 = 0
best_hyperparameters_resnet50 = {}

stop_tuning_early_resnet50 = False


for lr in learning_rates:
    if stop_tuning_early_resnet50:
        break
    for units in dense_units:
        if stop_tuning_early_resnet50:
            break
        for dropout in dropout_rates:
            if stop_tuning_early_resnet50:
                break

            print(f"\nAttempting training with LR: {lr}, Dense Units: {units}, Dropout: {dropout}")

            # Create a new model
            model_tuned_resnet50 = Sequential([
                base_model_resnet50_tune,
                layers.GlobalAveragePooling2D(),
                layers.Dense(units, activation='relu'),
                layers.Dropout(dropout),
                layers.Dense(train_dataset.num_classes, activation='softmax')
            ])

            # Compile the model
            model_tuned_resnet50.compile(optimizer=optimizers.Adam(learning_rate=lr),
                                         loss='categorical_crossentropy',
                                         metrics=['accuracy'])

            # Train the model
            epochs_tune = 1
            history_tuned_resnet50 = model_tuned_resnet50.fit(
                train_dataset,
                steps_per_epoch=train_dataset.samples // batch_size,
                epochs=epochs_tune,
                validation_data=val_dataset,
                validation_steps=val_dataset.samples // batch_size,
                verbose=0
            )

            # Evaluate on validation set
            val_loss, val_accuracy = model_tuned_resnet50.evaluate(val_dataset, verbose=0)
            print(f"Validation Accuracy for current trial: {val_accuracy:.4f}")

            if val_accuracy > best_accuracy_resnet50:
                best_accuracy_resnet50 = val_accuracy
                best_hyperparameters_resnet50 = {'learning_rate': lr, 'dense_units': units, 'dropout_rate': dropout}
                print("New best hyperparameters found!")
                stop_tuning_early_resnet50 = True

print("\nHyperparameter tuning complete (Simplified Example).")
print("Best Validation Accuracy:", best_accuracy_resnet50)
print("Best Hyperparameters:", best_hyperparameters_resnet50)

##### Which hyperparameter optimization technique have you used and why?

- A simplified manual grid search to explore hyperparameters for the layers added on top of the frozen ResNet50 base (Learning Rate, Dense Units, Dropout Rate). This approach illustrates tuning the classification part of a transfer learning model.

##### Have you seen any improvement? Note down the improvement with updates Evaluation metric Score Chart.

- In the limited tuning trials conducted, a validation accuracy of [Insert Best Validation Accuracy Here] was reached with hyperparameters: [Insert Best Hyperparameters Here]. This indicates the potential for performance changes by optimizing the added layers, subject to a full tuning process.

### ML Model - 4

In [ ]:
# ML Model - 4 Implementation (Transfer Learning with MobileNet)

# Load the pre-trained MobileNet model
base_model_mobilenet = MobileNet(weights='imagenet', include_top=False, input_shape=(img_size[0], img_size[1], 3))

for layer in base_model_mobilenet.layers:
    layer.trainable = False

# Create a new model
model_mobilenet = Sequential([
    base_model_mobilenet,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(train_dataset.num_classes, activation='softmax')
])

# Compile the model
model_mobilenet.compile(optimizer=optimizers.Adam(learning_rate=0.001),
                       loss='categorical_crossentropy',
                       metrics=['accuracy'])

# Display model summary
model_mobilenet.summary()

# Train the model
epochs_mobilenet = 5
history_mobilenet = model_mobilenet.fit(
    train_dataset,
    steps_per_epoch=train_dataset.samples // batch_size,
    epochs=epochs_mobilenet,
    validation_data=val_dataset,
    validation_steps=val_dataset.samples // batch_size
)

# Save the trained model
model_mobilenet.save('/content/drive/MyDrive/my_models/mobilenet_model.keras')
print("\nMobileNet model saved successfully.")

#### 1. Explain the ML Model used and it's performance using Evaluation metric Score Chart.

In [ ]:
# Visualizing evaluation Metric Score chart (for ML Model - 4)

# Load the saved MobileNet model
loaded_model_mobilenet = load_model('/content/drive/MyDrive/my_models/mobilenet_model.keras') # Use .keras as saved earlier
print("MobileNet model loaded successfully.")

# Evaluate the loaded model on the test dataset
print("\nEvaluating the loaded MobileNet model on the test dataset...")
loss_mobilenet, accuracy_mobilenet = loaded_model_mobilenet.evaluate(test_dataset)

print(f"\nTest Loss: {loss_mobilenet:.4f}")
print(f"Test Accuracy: {accuracy_mobilenet:.4f}")

test_predictions_mobilenet = loaded_model_mobilenet.predict(test_dataset, verbose=0)
test_predicted_labels_mobilenet = np.argmax(test_predictions_mobilenet, axis=1)

# Get true labels (assuming test_dataset.classes is available)
test_true_labels = test_dataset.classes

# Get class names (assuming train_dataset.class_indices is available)
class_names = list(train_dataset.class_indices.keys())

# Generate Classification Report
print("\nClassification Report (MobileNet Model):")
print(classification_report(test_true_labels, test_predicted_labels_mobilenet, target_names=class_names))

# Generate Confusion Matrix
print("\nConfusion Matrix (MobileNet Model):")
cm_mobilenet = confusion_matrix(test_true_labels, test_predicted_labels_mobilenet)

# Visualize Confusion Matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm_mobilenet, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix (MobileNet Model)')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# Visualize Training History
if 'history_mobilenet' in locals():
    plt.figure(figsize=(12, 6))

    # Plot training & validation accuracy values
    plt.subplot(1, 2, 1)
    plt.plot(history_mobilenet.history['accuracy'])
    plt.plot(history_mobilenet.history['val_accuracy'])
    plt.title('Model Accuracy (MobileNet)')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    # Plot training & validation loss values
    plt.subplot(1, 2, 2)
    plt.plot(history_mobilenet.history['loss'])
    plt.plot(history_mobilenet.history['val_loss'])
    plt.title('Model Loss (MobileNet)')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    plt.tight_layout()
    plt.show()
else:
    print("\nTraining history object 'history_mobilenet' not found. Cannot plot training history.")

#### 2. Cross- Validation & Hyperparameter Tuning

In [ ]:
# ML Model - 4 Hyperparameter Tuning (Transfer Learning with MobileNet)

# Load the pre-trained MobileNet base model
base_model_mobilenet_tune = MobileNet(weights='imagenet', include_top=False, input_shape=(img_size[0], img_size[1], 3))

for layer in base_model_mobilenet_tune.layers:
    layer.trainable = False

learning_rates = [0.001, 0.0005]
dense_units = [128, 256]
dropout_rates = [0.4, 0.5]

best_accuracy_mobilenet = 0
best_hyperparameters_mobilenet = {}

stop_tuning_early_mobilenet = False


for lr in learning_rates:
    if stop_tuning_early_mobilenet:
        break
    for units in dense_units:
        if stop_tuning_early_mobilenet:
            break
        for dropout in dropout_rates:
            if stop_tuning_early_mobilenet:
                break

            print(f"\nAttempting training with LR: {lr}, Dense Units: {units}, Dropout: {dropout}")

            model_tuned_mobilenet = Sequential([
                base_model_mobilenet_tune,
                layers.GlobalAveragePooling2D(),
                layers.Dense(units, activation='relu'),
                layers.Dropout(dropout),
                layers.Dense(train_dataset.num_classes, activation='softmax')
            ])

            # Compile the model
            model_tuned_mobilenet.compile(optimizer=optimizers.Adam(learning_rate=lr),
                                          loss='categorical_crossentropy',
                                          metrics=['accuracy'])

            # Train the model
            epochs_tune = 1
            history_tuned_mobilenet = model_tuned_mobilenet.fit(
                train_dataset,
                steps_per_epoch=train_dataset.samples // batch_size,
                epochs=epochs_tune,
                validation_data=val_dataset,
                validation_steps=val_dataset.samples // batch_size,
                verbose=0
            )

            # Evaluate on validation set
            val_loss, val_accuracy = model_tuned_mobilenet.evaluate(val_dataset, verbose=0)
            print(f"Validation Accuracy for current trial: {val_accuracy:.4f}")

            if val_accuracy > best_accuracy_mobilenet:
                best_accuracy_mobilenet = val_accuracy
                best_hyperparameters_mobilenet = {'learning_rate': lr, 'dense_units': units, 'dropout_rate': dropout}
                print("New best hyperparameters found!")
                stop_tuning_early_mobilenet = True

print("\nHyperparameter tuning complete (Simplified Example).")
print("Best Validation Accuracy:", best_accuracy_mobilenet)
print("Best Hyperparameters:", best_hyperparameters_mobilenet)

##### Which hyperparameter optimization technique have you used and why?

-  A simplified manual grid search was used for tuning hyperparameters (Learning Rate, Dense Units, Dropout Rate) in the layers added to the pre-trained MobileNet base. This serves as a basic example of optimizing the classification head in transfer learning.

##### Have you seen any improvement? Note down the improvement with updates Evaluation metric Score Chart.

- From the limited trials before early stopping, a validation accuracy of [Insert Best Validation Accuracy Here] was achieved with hyperparameters: [Insert Best Hyperparameters Here]. This suggests that tuning the top layers can influence performance, but a complete search is required for optimal results.

### ML Model - 5

In [ ]:
# ML Model - 5 Implementation (Transfer Learning with InceptionV3)

# Load the pre-trained model
base_model_inceptionv3 = InceptionV3(weights='imagenet', include_top=False, input_shape=(img_size[0], img_size[1], 3))

for layer in base_model_inceptionv3.layers:
    layer.trainable = False

# Create a new model
model_inceptionv3 = Sequential([
    base_model_inceptionv3,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(train_dataset.num_classes, activation='softmax')
])

# Compile the model
model_inceptionv3.compile(optimizer=optimizers.Adam(learning_rate=0.001),
                       loss='categorical_crossentropy',
                       metrics=['accuracy'])

# Display model summary
model_inceptionv3.summary()

# Train the model
epochs_inceptionv3 = 5
history_inceptionv3 = model_inceptionv3.fit(
    train_dataset,
    steps_per_epoch=train_dataset.samples // batch_size,
    epochs=epochs_inceptionv3,
    validation_data=val_dataset,
    validation_steps=val_dataset.samples // batch_size
)

# Save the trained model
model_inceptionv3.save('/content/drive/MyDrive/my_models/inceptionv3_model.keras')
print("\nInceptionV3 model saved successfully.")

#### 1. Explain the ML Model used and it's performance using Evaluation metric Score Chart.

In [ ]:
# Visualizing evaluation Metric Score chart (for ML Model - 5)

loaded_model_inceptionv3 = load_model('/content/drive/MyDrive/my_models/inceptionv3_model.keras')
print("InceptionV3 model loaded successfully.")

# Evaluate the loaded model
print("\nEvaluating the loaded InceptionV3 model on the test dataset...")
loss_inceptionv3, accuracy_inceptionv3 = loaded_model_inceptionv3.evaluate(test_dataset)

print(f"\nTest Loss: {loss_inceptionv3:.4f}")
print(f"Test Accuracy: {accuracy_inceptionv3:.4f}")

test_predictions_inceptionv3 = loaded_model_inceptionv3.predict(test_dataset, verbose=0)
test_predicted_labels_inceptionv3 = np.argmax(test_predictions_inceptionv3, axis=1)

# Get true labels
test_true_labels = test_dataset.classes

# Get class names
class_names = list(train_dataset.class_indices.keys())

# Generate Classification Report
print("\nClassification Report (InceptionV3 Model):")
print(classification_report(test_true_labels, test_predicted_labels_inceptionv3, target_names=class_names))

# Generate Confusion Matrix
print("\nConfusion Matrix (InceptionV3 Model):")
cm_inceptionv3 = confusion_matrix(test_true_labels, test_predicted_labels_inceptionv3)

# Visualize Confusion Matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm_inceptionv3, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix (InceptionV3 Model)')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

# Visualize Training History
if 'history_inceptionv3' in locals():
    plt.figure(figsize=(12, 6))

    # Plot training & validation accuracy values
    plt.subplot(1, 2, 1)
    plt.plot(history_inceptionv3.history['accuracy'])
    plt.plot(history_inceptionv3.history['val_accuracy'])
    plt.title('Model Accuracy (InceptionV3)')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    # Plot training & validation loss values
    plt.subplot(1, 2, 2)
    plt.plot(history_inceptionv3.history['loss'])
    plt.plot(history_inceptionv3.history['val_loss'])
    plt.title('Model Loss (InceptionV3)')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    plt.tight_layout()
    plt.show()
else:
    print("\nTraining history object 'history_inceptionv3' not found. Cannot plot training history.")

#### 2. Cross- Validation & Hyperparameter Tuning

In [ ]:
# ML Model - 5 Hyperparameter Tuning (Transfer Learning with InceptionV3 - Simplified Example)

# Load the pre-trained model
base_model_inceptionv3_tune = InceptionV3(weights='imagenet', include_top=False, input_shape=(img_size[0], img_size[1], 3))

for layer in base_model_inceptionv3_tune.layers:
    layer.trainable = False

learning_rates = [0.001, 0.0005]
dense_units = [128, 256]
dropout_rates = [0.4, 0.5]

best_accuracy_inceptionv3 = 0
best_hyperparameters_inceptionv3 = {}

stop_tuning_early_inceptionv3 = False


for lr in learning_rates:
    if stop_tuning_early_inceptionv3:
        break
    for units in dense_units:
        if stop_tuning_early_inceptionv3:
            break
        for dropout in dropout_rates:
            if stop_tuning_early_inceptionv3:
                break

            print(f"\n train with LR: {lr}, Dense Units: {units}, Dropout: {dropout}")

            model_tuned_inceptionv3 = Sequential([
                base_model_inceptionv3_tune,
                layers.GlobalAveragePooling2D(),
                layers.Dense(units, activation='relu'),
                layers.Dropout(dropout),
                layers.Dense(train_dataset.num_classes, activation='softmax')
            ])

            model_tuned_inceptionv3.compile(optimizer=optimizers.Adam(learning_rate=lr),
                                         loss='categorical_crossentropy',
                                         metrics=['accuracy'])

            epochs_tune = 1
            history_tuned_inceptionv3 = model_tuned_inceptionv3.fit(
                train_dataset,
                steps_per_epoch=train_dataset.samples // batch_size,
                epochs=epochs_tune,
                validation_data=val_dataset,
                validation_steps=val_dataset.samples // batch_size,
                verbose=0
            )

            # Evaluate on validation set
            val_loss, val_accuracy = model_tuned_inceptionv3.evaluate(val_dataset, verbose=0)
            print(f"Validation Accuracy for current trial: {val_accuracy:.4f}")

            if val_accuracy > best_accuracy_inceptionv3:
                best_accuracy_inceptionv3 = val_accuracy
                best_hyperparameters_inceptionv3 = {'learning_rate': lr, 'dense_units': units, 'dropout_rate': dropout}
                print("New best hyperparameters found!")

                stop_tuning_early_inceptionv3 = True

print("\nHyperparameter tuning complete (Simplified Example).")
print("Best Validation Accuracy:", best_accuracy_inceptionv3)
print("Best Hyperparameters:", best_hyperparameters_inceptionv3)

##### Which hyperparameter optimization technique have you used and why?

- A simplified manual grid search was applied to the hyperparameters of the newly added layers on top of the pre-trained InceptionV3 base (Learning Rate, Dense Units, Dropout Rate). This method helps demonstrate tuning the classification head in a transfer learning setup.


##### Have you seen any improvement? Note down the improvement with updates Evaluation metric Score Chart.

- From the limited tuning trials, a validation accuracy of [Insert Best Validation Accuracy Here] was achieved with hyperparameters: [Insert Best Hyperparameters Here]. This indicates potential gains with tuning the top layers.

### ML Model - 6

In [ ]:
# ML Model - 6 Implementation (Transfer Learning with EfficientNetB0)

# Load the pre-trained model
base_model_efficientnet = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(img_size[0], img_size[1], 3))

for layer in base_model_efficientnet.layers:
    layer.trainable = False

model_efficientnet = Sequential([
    base_model_efficientnet,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(train_dataset.num_classes, activation='softmax')
])

# Compile the model
model_efficientnet.compile(optimizer=optimizers.Adam(learning_rate=0.001),
                       loss='categorical_crossentropy',
                       metrics=['accuracy'])

# Display model summary
model_efficientnet.summary()

# Train the model
epochs_efficientnet = 5
history_efficientnet = model_efficientnet.fit(
    train_dataset,
    steps_per_epoch=train_dataset.samples // batch_size,
    epochs=epochs_efficientnet,
    validation_data=val_dataset,
    validation_steps=val_dataset.samples // batch_size
)

# Save the trained model
model_efficientnet.save('/content/drive/MyDrive/my_models/efficientnet_model.keras')
print("\nEfficientNetB0 model saved successfully.")

#### 1. Explain the ML Model used and it's performance using Evaluation metric Score Chart.

In [ ]:
# Visualizing evaluation Metric Score chart (for ML Model - 6)

loaded_model_efficientnet = load_model('/content/drive/MyDrive/my_models/efficientnet_model.keras')
print("EfficientNetB0 model loaded successfully.")

print("\nEvaluation of the loaded EfficientNetB0 model on the test dataset")
loss_efficientnet, accuracy_efficientnet = loaded_model_efficientnet.evaluate(test_dataset)

print(f"\nTest Loss: {loss_efficientnet:.4f}")
print(f"Test Accuracy: {accuracy_efficientnet:.4f}")

test_predictions_efficientnet = loaded_model_efficientnet.predict(test_dataset, verbose=0)
test_predicted_labels_efficientnet = np.argmax(test_predictions_efficientnet, axis=1)

test_true_labels = test_dataset.classes

class_names = list(train_dataset.class_indices.keys())

# Generate Classification Report
print("\nClassification Report (EfficientNetB0 Model):")
print(classification_report(test_true_labels, test_predicted_labels_efficientnet, target_names=class_names))

# Generate Confusion Matrix
print("\nConfusion Matrix (EfficientNetB0 Model):")
cm_efficientnet = confusion_matrix(test_true_labels, test_predicted_labels_efficientnet)

# Visualize Confusion Matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm_efficientnet, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix (EfficientNetB0 Model)')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()


if 'history_efficientnet' in locals():
    plt.figure(figsize=(12, 6))

    # Plot training & validation accuracy values
    plt.subplot(1, 2, 1)
    plt.plot(history_efficientnet.history['accuracy'])
    plt.plot(history_efficientnet.history['val_accuracy'])
    plt.title('Model Accuracy (EfficientNetB0)')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    # Plot training & validation loss values
    plt.subplot(1, 2, 2)
    plt.plot(history_efficientnet.history['loss'])
    plt.plot(history_efficientnet.history['val_loss'])
    plt.title('Model Loss (EfficientNetB0)')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')

    plt.tight_layout()
    plt.show()
else:
    print("\nTraining history object 'history_efficientnet' not found. Cannot plot training history.")

#### 2. Cross- Validation & Hyperparameter Tuning

In [ ]:
# ML Model - 6 Hyperparameter Optimization (Transfer Learning with EfficientNetB0)

# Load the pretrained model
base_model_efficientnet_tune = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(img_size[0], img_size[1], 3))

for layer in base_model_efficientnet_tune.layers:
    layer.trainable = False

learning_rates = [0.001, 0.0005]
dense_units = [128, 256]
dropout_rates = [0.4, 0.5]

best_accuracy_efficientnet = 0
best_hyperparameters_efficientnet = {}

stop_tuning_early_efficientnet = False


for lr in learning_rates:
    if stop_tuning_early_efficientnet:
        break
    for units in dense_units:
        if stop_tuning_early_efficientnet:
            break
        for dropout in dropout_rates:
            if stop_tuning_early_efficientnet:
                break

            print(f"\n train with LR: {lr}, Dense Units: {units}, Dropout: {dropout}")


            model_tuned_efficientnet = Sequential([
                base_model_efficientnet_tune,
                layers.GlobalAveragePooling2D(),
                layers.Dense(units, activation='relu'),
                layers.Dropout(dropout),
                layers.Dense(train_dataset.num_classes, activation='softmax')
            ])

            model_tuned_efficientnet.compile(optimizer=optimizers.Adam(learning_rate=lr),
                                         loss='categorical_crossentropy',
                                         metrics=['accuracy'])


            epochs_tune = 1
            history_tuned_efficientnet = model_tuned_efficientnet.fit(
                train_dataset,
                steps_per_epoch=train_dataset.samples // batch_size,
                epochs=epochs_tune,
                validation_data=val_dataset,
                validation_steps=val_dataset.samples // batch_size,
                verbose=0
            )

            val_loss, val_accuracy = model_tuned_efficientnet.evaluate(val_dataset, verbose=0)
            print(f"Validation Accuracy for current trial: {val_accuracy:.4f}")

            if val_accuracy > best_accuracy_efficientnet:
                best_accuracy_efficientnet = val_accuracy
                best_hyperparameters_efficientnet = {'learning_rate': lr, 'dense_units': units, 'dropout_rate': dropout}
                print("New best hyperparameters found!")
                stop_tuning_early_efficientnet = True

print("\nHyperparameter tuning complete (Simplified Example).")
print("Best Validation Accuracy:", best_accuracy_efficientnet)
print("Best Hyperparameters:", best_hyperparameters_efficientnet)

##### Which hyperparameter optimization technique have you used and why?

- We employed a simplified manual grid search to explore hyperparameters for the layers added on top of the frozen EfficientNetB0 base (Learning Rate, Dense Units, Dropout Rate). This approach illustrates tuning the classification part of a transfer learning model.

##### Have you seen any improvement? Note down the improvement with updates Evaluation metric Score Chart.

- In the limited tuning trials conducted, a validation accuracy of [Insert Best Validation Accuracy Here] was reached with hyperparameters: [Insert Best Hyperparameters Here]. This indicates the potential for performance changes by optimizing the added layers, subject to a full tuning process.

#### Explain each evaluation metric's indication towards business and the business impact pf the ML model used.

- **Evaluating the performance of a machine learning model is essential for understanding its effectiveness and its alignment with business objectives. In the context of a multiclass classification model, such as a fish classifier, several key metrics provide varied perspectives on performance:**

 * **Accuracy:**
    * **Implication for Business:** Accuracy reflects the overall correctness of the model, measuring the proportion of correct predictions across all classes.
    * **Business Impact:** High accuracy indicates that the model is generally proficient at identifying fish species. In a business context, such as automated sorting or monitoring, high accuracy translates to fewer errors, enhancing operational efficiency and diminishing the necessity for manual verification. However, accuracy can be misleading in cases of imbalanced datasets.

 * **Precision:**
    * **Implication for Business:** Precision assesses the accuracy of positive predictions for a specific class, calculated as the ratio of true positives to the total predicted positives (True Positives + False Positives). It addresses the question: "Of all instances where the model predicted this fish species, how many were accurate?"
    * **Business Impact:** High precision signifies a lower occurrence of "false alarms" or incorrect identifications for that particular species. This metric is particularly significant when the cost of a false positive is substantial, such as mislabeling a protected species, which could result in regulatory complications, or incorrectly categorizing a high-value fish as low-value.

 * **Recall (Sensitivity):**
    * **Implication for Business:** Recall measures the model's capacity to identify all actual positive instances for a specific class, represented by the ratio of true positives to the total actual positives (True Positives + False Negatives). It answers the inquiry: "Of all actual images of this fish species, how many did the model correctly classify?"
    * **Business Impact:** High recall becomes crucial when the cost of a false negative is elevated, such as failing to detect an invasive species, overlooking a significant fish during a survey, or not identifying a diseased fish. It ensures comprehensive capture of instances within a particular class.

 * **F1-Score:**
    * **Implication for Business:** The F1-score is the harmonic mean of Precision and Recall, providing a single metric that balances both considerations. This measure is particularly advantageous in scenarios with uneven class distribution.
    * **Business Impact:** The F1-score presents a holistic view of performance, particularly valuable when both false positives and false negatives carry noteworthy consequences. A high F1-score indicates that the model successfully maintains a balance between accurately identifying positive cases and minimizing the occurrence of false positives.

 * **Confusion Matrix:**
    * **Implication for Business:** A confusion matrix visualizes the performance of a classification model on a designated test dataset, displaying counts of true positives, true negatives, false positives, and false negatives for each class.
    * **Business Impact:** This matrix provides an intricate breakdown of the model’s successes and failures. It delineates which classes are often mistaken for others (e.g., misclassifying one fish species as another), thus offering critical insight for identifying specific problem areas that may require additional data, model refinements, or in-depth analyses. This understanding directly influences the reliability and trustworthiness of the model within a business application.

- **Overall Business Impact:**

  The effective implementation of this machine learning model holds substantial positive implications for business operations:

 * **Increased Efficiency:** The automation of fish identification significantly diminishes manual labor while expediting processes such as sorting, monitoring, and data collection.
 * **Improved Accuracy:** A well-functioning model can attain a higher level of consistency and potentially superior accuracy compared to manual methods, thereby reducing errors.
 * **Cost Reduction:** The decrease in manual effort and a reduction in errors can lead to lower operational expenses.
 * **Enhanced Monitoring and Compliance:** Accurate identification facilitates improved monitoring of fish populations, aids in the enforcement of fishing regulations, and supports biodiversity tracking.
 * **Data-Driven Decision Making:** The model supplies valuable insights regarding species composition, which can inform management decisions and contribute to research initiatives.


### 1. Which Evaluation metrics did you consider for a positive business impact and why?

- While overall **Accuracy** provides a general sense of the model's performance, for a positive business impact in fish classification, metrics that provide a more nuanced view of correct identifications and errors are particularly important. Therefore, we primarily considered **Precision**, **Recall**, and the **F1-score**.

 *   **Precision** is crucial because a high number of false positives (misidentifying a fish) can lead to significant costs, such as incorrect data collection, mislabeling of products, or even regulatory issues if protected species are wrongly identified. High precision minimizes these costly false alarms.
 *   **Recall** is equally important because a high number of false negatives (failing to identify a fish that is present) can result in missed opportunities (e.g., failing to count a valuable species), missed detection of problems (e.g., invasive species or disease), or incomplete data for monitoring. High recall ensures that most relevant instances are captured.
 *   The **F1-score** provides a balanced measure of both Precision and Recall. In most real-world scenarios for fish classification, both false positives and false negatives carry significant consequences. The F1-score helps in finding a model that achieves a good balance between minimizing both types of errors, leading to a more reliable and impactful system.

- By focusing on these metrics, we can select a model that not only gets predictions right overall (accuracy) but also specifically manages the types of errors that are most costly or detrimental to the business or conservation goals.

### 2. Which ML model did you choose from the above created models as your final prediction model and why?

In [ ]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import load_model

img_size = (224, 224)

model_paths = {
    'CNN': '/content/drive/MyDrive/my_models/cnn_model.keras',
    'VGG16': '/content/drive/MyDrive/my_models/vgg16_model.keras',
    'ResNet50': '/content/drive/MyDrive/my_models/resnet50_model.keras',
    'MobileNet': '/content/drive/MyDrive/my_models/mobilenet_model.keras',
    'InceptionV3': '/content/drive/MyDrive/my_models/inceptionv3_model.keras',
    'EfficientNetB0': '/content/drive/MyDrive/my_models/efficientnet_model.keras'
}

try:
    sample_image_info = test_df.sample(1, random_state=42).iloc[0]
    sample_image_path = sample_image_info['filepath']
    true_label = sample_image_info['label']
    print(f"Using sample unseen image: {sample_image_path}")
    print(f"True label: {true_label}")

    img = Image.open(sample_image_path)
    img = img.resize(img_size)
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Display the sample image
    plt.imshow(img)
    plt.title(f"Sample Image (True: {true_label})")
    plt.axis('off')
    plt.show()

    class_names = list(train_dataset.class_indices.keys())

    print("\n--- Predictions for Sample Image ---")

    model_accuracies = {}

    for model_name, model_path in model_paths.items():
        try:
            # Load the model
            model = load_model(model_path)

            # Make prediction
            predictions = model.predict(img_array, verbose=0)
            predicted_class_index = np.argmax(predictions)
            confidence_score = predictions[0][predicted_class_index]
            predicted_label = class_names[predicted_class_index]

            print(f"{model_name} Prediction: {predicted_label} (Confidence: {confidence_score:.4f})")

            if model_name == 'CNN' and 'accuracy_cnn' in locals():
                model_accuracies[model_name] = accuracy_cnn
            elif model_name == 'VGG16' and 'accuracy_vgg16' in locals():
                 model_accuracies[model_name] = accuracy_vgg16
            elif model_name == 'ResNet50' and 'accuracy_resnet50' in locals():
                 model_accuracies[model_name] = accuracy_resnet50
            elif model_name == 'MobileNet' and 'accuracy_mobilenet' in locals():
                 model_accuracies[model_name] = accuracy_mobilenet
            elif model_name == 'InceptionV3' and 'accuracy_inceptionv3' in locals():
                 model_accuracies[model_name] = accuracy_inceptionv3
            elif model_name == 'EfficientNetB0' and 'accuracy_efficientnet' in locals():
                 model_accuracies[model_name] = accuracy_efficientnet
            else:
                 model_accuracies[model_name] = "N/A (Run evaluation cell)"


        except FileNotFoundError:
            print(f"Error: Model file not found for {model_name} at {model_path}")
            model_accuracies[model_name] = "Error (File Not Found)"
        except Exception as e:
            print(f"An error occurred during prediction with {model_name}: {e}")
            model_accuracies[model_name] = "Error (Prediction Failed)"

    print("\n--- Test Accuracy Comparison ---")
    best_model_name = None
    highest_accuracy = -1

    for name, acc in model_accuracies.items():
        if isinstance(acc, (int, float)):
            print(f"{name}: {acc:.4f}")
            if acc > highest_accuracy:
                highest_accuracy = acc
                best_model_name = name
        else:
            print(f"{name}: {acc}")


    if best_model_name and highest_accuracy != -1:
        print(f"\nBest performing model based on Test Accuracy: {best_model_name} ({highest_accuracy:.4f})")
    else:
        print("\nCould not determine the best performing model based on available test accuracies.")


except FileNotFoundError:
    print(f"Error: The sample image file was not found at {sample_image_path}. Please ensure the file path is correct.")
except NameError as e:
    print(f"Error: Required variables (e.g., test_df, train_dataset) are not defined. Please run the data loading and preprocessing cells.")
    print(f"Specific error: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

- Based on the comparative evaluation of all implemented ML models in Section 6, the **MobileNet (ML Model 4)** was chosen as the final prediction model for deployment.

- The primary reason for selecting MobileNet was its superior performance on the test dataset, achieving the highest overall accuracy compared to the CNN from scratch, VGG16, ResNet50, InceptionV3, and EfficientNetB0 models under similar training conditions. The evaluation metrics, including the classification report and confusion matrix, further demonstrated MobileNet's effectiveness in correctly classifying most fish species with high confidence.

- Additionally, MobileNet is known for being a relatively lightweight and efficient architecture, which is advantageous for deployment scenarios, especially if the application needs to run on devices with limited computational resources. While other models like VGG16 and InceptionV3 also performed well, MobileNet provided the best balance of accuracy and efficiency among the experimented models for this specific task.

## ***7.*** ***Future Work***

### 1. Save the best performing ml model in a pickle file or joblib file format for deployment process.


In [ ]:
# Save the File
best_model = load_model('/content/drive/MyDrive/my_models/mobilenet_model.keras')
print("Best model (MobileNet) loaded successfully.")

best_model_save_path = '/content/drive/MyDrive/my_models/best_fish_classifier_model.keras'

# Save the best model
best_model.save(best_model_save_path)
print(f"\nBest model saved successfully to: {best_model_save_path}")

### 2. Again Load the saved model file and try to predict unseen data for a sanity check.


In [ ]:
# Load the File and predict unseen data.

best_model_load_path = '/content/drive/MyDrive/my_models/best_fish_classifier_model.keras'

loaded_best_model = load_model(best_model_load_path)
print(f"Best model loaded successfully from: {best_model_load_path}")

sample_image_info = test_df.sample(1, random_state=42).iloc[0]
sample_image_path = sample_image_info['filepath']
true_label = sample_image_info['label']
print(f"\nLoading sample unseen image: {sample_image_path}")
print(f"True label: {true_label}")

# Load and preprocess the sample image
try:
    img = Image.open(sample_image_path)
    img = img.resize(img_size)
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    # Make a prediction
    predictions = loaded_best_model.predict(img_array)

    predicted_class_index = np.argmax(predictions)
    confidence_score = predictions[0][predicted_class_index]

    class_names = list(train_dataset.class_indices.keys())
    predicted_label = class_names[predicted_class_index]

    print(f"\nPredicted label: {predicted_label}")
    print(f"Confidence score: {confidence_score:.4f}")

    plt.imshow(img)
    plt.title(f"True: {true_label}\nPredicted: {predicted_label} ({confidence_score:.2f})")
    plt.axis('off')
    plt.show()

except FileNotFoundError:
    print(f"Error: The sample image file was not found at {sample_image_path}. Please ensure the file path is correct.")
except Exception as e:
    print(f"An error occurred during prediction: {e}")

## Streamlit Deployment

In [ ]:
# Install dependencies
!pip install streamlit ngrok pyngrok

In [ ]:
%%writefile app.py
import streamlit as st
import tensorflow as tf
import numpy as np
from PIL import Image
import json
import os

# Define image size expected by the model
img_size = (224, 224)

# Define the path to the saved best model
best_model_load_path = '/content/drive/MyDrive/my_models/best_fish_classifier_model.keras'

class_indices_path = '/content/drive/MyDrive/my_data/class_indices.json'

# Load the saved model
@st.cache_resource
def load_model(model_path):
    """Loads the pre-trained Keras model."""
    try:
        model = tf.keras.models.load_model(model_path)
        return model
    except Exception as e:
        st.error(f"Error loading model: {e}")
        return None

# Load class indices mapping
@st.cache_resource
def load_class_indices(indices_path):
    """Loads the class indices mapping from a JSON file."""
    try:
        with open(indices_path, 'r') as f:
            class_indices = json.load(f)
        # Invert the dictionary to get index -> class name mapping
        return {v: k for k, v in class_indices.items()}
    except FileNotFoundError:
        st.error(f"Class indices file not found at {indices_indices_path}")
        return None
    except Exception as e:
        st.error(f"Error loading class indices: {e}")
        return None


model = load_model(best_model_load_path)
idx_to_class = load_class_indices(class_indices_path)

# Streamlit application layout
st.title("Multiclass Fish Image Classification")

st.write("Upload an image of a fish to classify it.")

uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "jpeg", "png"])

if uploaded_file is not None:
    try:
        # Display the uploaded image
        image = Image.open(uploaded_file)
        st.image(image, caption="Uploaded Image.", use_column_width=True)

        # Preprocess the image for prediction
        img = image.resize(img_size)
        img_array = np.array(img) / 255.0
        img_array = np.expand_dims(img_array, axis=0) # Add batch dimension

        if model is not None and idx_to_class is not None:
            # Make prediction
            st.write("Classifying...")
            predictions = model.predict(img_array)
            predicted_class_index = np.argmax(predictions)
            confidence_score = predictions[0][predicted_class_index]

            # Get predicted label
            predicted_label = idx_to_class.get(predicted_class_index, "Unknown")

            # Display results
            st.write(f"Prediction: **{predicted_label}**")
            st.write(f"Confidence: **{confidence_score:.2f}**")
        elif model is None:
            st.error("Model could not be loaded. Please check the model path.")
        else: # idx_to_class is None
             st.error("Class indices could not be loaded. Cannot interpret prediction results.")

    except Exception as e:
        st.error(f"An error occurred during image processing or prediction: {e}")

In [ ]:
# Set up ngrok tunnel by querying the local API

import os
import time
import requests
import json

auth_token = "2zfEFhgAORGwzbHpkesKDMBDyx5_nmnrB3QYJTr46jy9HhZ8"
try:
    ngrok_config_path = f"{os.path.expanduser('~')}/.ngrok2/ngrok.yml"
    config_content = ""
    if os.path.exists(ngrok_config_path):
        with open(ngrok_config_path, 'r') as f:
            config_content = f.read()

    if auth_token not in config_content:
        print("Authenticating ngrok...")
        !ngrok authtoken {auth_token}
        print("ngrok authentication complete.")
    else:
        print("ngrok authentication token already set.")

except Exception as e:
    print(f"Error during ngrok authentication setup: {e}")

try:
    print("\nStarting ngrok tunnel to port 8501 in the background...")
    !pkill ngrok > /dev/null 2>&1
    get_ipython().system_raw('ngrok http 8501 &')
    print("ngrok agent started in the background.")

except Exception as e:
    print(f"Error starting ngrok agent: {e}")


print("Waiting for ngrok tunnel to establish...")
time.sleep(5)


ngrok_api_url = "http://localhost:4040/api/tunnels"
public_url = None

try:
    response = requests.get(ngrok_api_url)
    response.raise_for_status()
    tunnel_data = response.json()


    for tunnel in tunnel_data['tunnels']:
        if tunnel['proto'] == 'http':
            public_url = tunnel['public_url']
            break

    if public_url:
        print(f"\nNgrok public URL: {public_url}")
    else:
        print("\nError: Could not find the ngrok public URL from the API.")
        print("Tunnel data:", tunnel_data)

except requests.exceptions.RequestException as e:
    print(f"\nError querying ngrok API: {e}")
    print("Please check if the ngrok agent started correctly and is running.")
except Exception as e:
    print(f"\nAn unexpected error occurred: {e}")

In [ ]:
# run streamlit app
!streamlit run app.py

# **Conclusion**

- This project successfully delivered a comprehensive end-to-end deep learning pipeline for multiclass fish image classification. The process began with the extraction and preprocessing of the raw dataset, progressed through exploratory data analysis (EDA), model building, and evaluation, and culminated in deployment within a Streamlit web application.

 **Key Achievements Include:**

  1. **Data Preparation & Quality Assurance:**
   The dataset underwent rigorous validation to ensure the absence of missing, duplicate, or corrupt images. Standardization to a consistent input size was implemented, and data augmentation techniques were utilized to enhance the model's robustness against variations in pose, lighting, and background.

  2. **Comprehensive EDA:**
   A series of over a dozen visualizations were generated to provide insights into class distribution, image properties, and feature characteristics, such as brightness and color variations. This thorough EDA facilitated informed modeling decisions and helped clarify potential challenges and opportunities within the dataset.

  3. **Model Experimentation:**
   A custom Convolutional Neural Network (CNN) was architected from scratch, alongside the implementation of five transfer learning architectures: VGG16, ResNet50, MobileNet, InceptionV3, and EfficientNetB0. Fine-tuning strategies were applied, and model training was optimized through the use of callbacks, including early stopping and learning rate reduction.

  4. **Evaluation & Selection:**
   A comprehensive evaluation of all models was conducted using various metrics: accuracy, precision, recall, F1-score, and confusion matrices. Through this analysis, the best-performing model was identified, showcasing high accuracy and balanced class performance, and was subsequently saved for deployment.

  5. **Explainability & Deployment:**
   The project integrated Grad-CAM for visual model interpretation, allowing for insights into the model's decision-making process. The chosen model was deployed as a Streamlit web application, providing real-time user predictions accompanied by confidence scores.

- **Final Outcome:**
The solution demonstrates that transfer learning, when paired with meticulously tailored fine-tuning, can substantially outperform a CNN developed from scratch in the context of visual classification tasks. The deployed application serves as an intuitive and accessible tool for instant fish species recognition, offering valuable applications in fisheries management, marine research, and the seafood industry.

- **Future Improvements Could Include:**
   - **Expanding the Dataset:** Enhancing the dataset by incorporating more species and diverse image conditions could improve model generalization.
   - **Advanced Hyperparameter Tuning and Ensemble Learning:** Implementing more sophisticated hyperparameter tuning and experimenting with ensemble learning techniques could further enhance model performance.
   - **Domain-Specific Augmentation:** Exploring domain-specific augmentation techniques, such as underwater blur simulation, may improve robustness in real-world scenarios.
   - **Cloud Deployment:** Transitioning the model to a production cloud environment could facilitate broader access and scalability.

- Overall, this project exemplifies the full machine learning lifecycle—data acquisition, model development, evaluation, and deployment—effectively showcasing a practical, accurate, and interpretable deep learning solution primed for real-world application.